## Import Library

In [1]:
# Import library dasar
import pandas as pd
import numpy as np

# Untuk similarity
from sklearn.metrics.pairwise import cosine_similarity

# Untuk normalisasi fitur numerik
from sklearn.preprocessing import MinMaxScaler

# Untuk visualisasi (opsional)
import matplotlib.pyplot as plt
import seaborn as sns

Pada tahap awal proyek, kita mengimpor beberapa library penting yang akan digunakan:

- **pandas** dan **numpy** untuk manipulasi data dan perhitungan numerik.
- **cosine_similarity** dari scikit-learn untuk menghitung kemiripan antar lagu berdasarkan fitur.
- **MinMaxScaler** untuk melakukan normalisasi fitur numerik agar berada dalam rentang yang sama.
- **matplotlib** dan **seaborn** sebagai library visualisasi data (opsional) untuk membantu eksplorasi data dan analisis.

Library ini adalah fondasi dasar yang akan mendukung seluruh proses pembangunan sistem rekomendasi.

## Memuat dan Mengeksplorasi Dataset

In [2]:
# Load dataset (ubah path sesuai file yang kamu download)
df = pd.read_csv('/content/drive/MyDrive/Membuat Model Sistem Rekomendasi/SpotifyFeatures.csv', nrows=10000)

# Lihat 5 data teratas
print(df.head())

# Cek info dataset
print(df.info())

# Cek missing values
print(df.isnull().sum())

   genre        artist_name                        track_name  \
0  Movie     Henri Salvador       C'est beau de faire un Show   
1  Movie  Martin & les fées  Perdu d'avance (par Gad Elmaleh)   
2  Movie    Joseph Williams    Don't Let Me Be Lonely Tonight   
3  Movie     Henri Salvador    Dis-moi Monsieur Gordon Cooper   
4  Movie       Fabien Nataf                         Ouverture   

                 track_id  popularity  acousticness  danceability  \
0  0BRjO6ga9RKCKjfDqeFgWV           0         0.611         0.389   
1  0BjC1NfoEOOusryehmNudP           1         0.246         0.590   
2  0CoSDzoNIKCRs124s9uTVy           3         0.952         0.663   
3  0Gc6TVm52BwZD07Ki6tIvf           0         0.703         0.240   
4  0IuslXpMROHdEPvSl1fTQK           4         0.950         0.331   

   duration_ms  energy  instrumentalness key  liveness  loudness   mode  \
0        99373   0.910             0.000  C#    0.3460    -1.828  Major   
1       137373   0.737             0.000  F#

Pada tahap ini, dataset lagu Spotify dimuat menggunakan `pandas` dengan fungsi `read_csv()`. Untuk mempercepat proses dan menghemat memori, kita hanya menggunakan 10.000 baris data dari file CSV.

Setelah memuat data, kita melakukan beberapa eksplorasi awal:
- Menampilkan 5 data teratas untuk melihat struktur dan contoh isi dataset.
- Mengecek informasi dataset dengan `info()` untuk mengetahui jumlah data, tipe data tiap kolom, dan penggunaan memori.
- Memeriksa missing values di seluruh kolom menggunakan `isnull().sum()` untuk memastikan tidak ada data yang hilang.

Hasil eksplorasi menunjukkan dataset berisi 10.000 lagu dengan 18 kolom fitur, tanpa missing value, dan berbagai tipe data seperti numerik dan kategorikal.

## Pemilihan Fitur untuk Sistem Rekomendasi

In [3]:
# Fitu yang digunakan
features = ['danceability', 'energy', 'valence', 'tempo', 'loudness']

df_selected = df[features]


Dalam membangun sistem rekomendasi berbasis content-based filtering, kita memilih beberapa fitur audio yang merepresentasikan karakteristik lagu, yaitu:

- **danceability**: Seberapa cocok lagu untuk berdansa.
- **energy**: Intensitas dan aktivitas energi lagu.
- **valence**: Positivitas atau mood lagu.
- **tempo**: Kecepatan lagu dalam BPM (beats per minute).
- **loudness**: Keras atau lembutnya lagu dalam desibel.

Fitur-fitur ini dipilih karena mampu merefleksikan atribut musik yang dapat membedakan satu lagu dengan lagu lainnya secara konten. Kemudian kita mengambil subset dataset hanya pada kolom fitur tersebut untuk proses selanjutnya.


## Normalisasi Fitur Numerik

In [4]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_selected)

df_scaled = pd.DataFrame(df_scaled, columns=features)


Karena fitur-fitur audio memiliki skala yang berbeda-beda (misalnya tempo dalam BPM, loudness dalam desibel), kita melakukan normalisasi menggunakan Min-Max Scaler.

Proses ini akan mengubah nilai fitur ke rentang 0 sampai 1 agar semua fitur memiliki bobot yang seimbang saat menghitung kemiripan (similarity) antar lagu.

Setelah proses normalisasi, data fitur siap untuk digunakan dalam perhitungan similarity.


## Menghitung Cosine Similarity antar Lagu

In [5]:
# Hitung cosine similarity antar lagu berdasarkan fitur
similarity_matrix = cosine_similarity(df_scaled)

# Uji coba tampilkan similarity pertama dengan lainnya
print(similarity_matrix[0][:5])

[1.         0.9844508  0.81498227 0.90025618 0.89758349]



Setelah fitur dinormalisasi, kita menghitung kemiripan antar lagu menggunakan *cosine similarity*.

Cosine similarity mengukur seberapa mirip dua lagu berdasarkan fitur audio mereka, dengan nilai 1 berarti sangat mirip (atau sama), dan nilai 0 berarti tidak mirip sama sekali.

Output contoh berupa nilai similarity antara lagu pertama dengan lagu-lagu lainnya, menunjukkan tingkat kemiripan fitur audio antar lagu.

## Membuat Fungsi Rekomendasi Lagu

In [6]:
# Tambahkan index judul lagu
df['track_name'] = df['track_name'].str.lower()  # lowercase agar pencarian mudah

# Buat mapping nama lagu ke index
track_to_idx = pd.Series(df.index, index=df['track_name'])

def recommend_songs(song_name, n=5):
    song_name = song_name.lower()
    if song_name not in track_to_idx:
        return f"Lagu '{song_name}' tidak ditemukan."

    idx = track_to_idx[song_name]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n+1]  # exclude lagu itu sendiri

    song_indices = [i[0] for i in sim_scores]

    return df.loc[song_indices, ['track_name', 'artist_name']]


Untuk memudahkan pencarian dan rekomendasi, kita lakukan langkah berikut:

1. Mengubah nama lagu menjadi huruf kecil (`lowercase`) agar pencarian lebih konsisten dan mudah.
2. Membuat *mapping* dari nama lagu ke indeks baris dataset untuk akses cepat.
3. Membuat fungsi `recommend_songs` yang:
   - Menerima nama lagu sebagai input.
   - Mencari indeks lagu tersebut.
   - Mengambil skor kemiripan lagu lain berdasarkan cosine similarity.
   - Mengurutkan lagu berdasarkan skor similarity tertinggi (kecuali lagu yang dicari sendiri).
   - Mengembalikan daftar rekomendasi lagu (nama dan artis).

Fungsi ini bisa digunakan untuk memberikan rekomendasi lagu yang mirip berdasarkan fitur audio.

## Contoh Penggunaan Fungsi Rekomendasi

In [7]:
recommend_songs('Shape of You', n=5)

,track_name,artist_name
1896,given up,Linkin Park
3706,god's been good to me,Keith Urban
2601,country song,Seether
660,29 ways to my back door,Tonic Sol-Fa
2192,bigger,Sugarland



Misalnya, kita ingin merekomendasikan 5 lagu yang mirip dengan lagu **"Shape of You"**.

Dengan memanggil fungsi:

```python
recommend_songs('Shape of You', n=5)

## Evaluasi Sistem Rekomendasi dengan Precision@K

In [8]:
def precision_at_k(song_name, k=5):
    song_name = song_name.lower()
    if song_name not in track_to_idx:
        return f"Lagu '{song_name}' tidak ditemukan."

    idx = track_to_idx[song_name]
    genre_input = df.loc[idx, 'genre']

    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:k+1]  # exclude lagu itu sendiri

    song_indices = [i[0] for i in sim_scores]
    recommended_genres = df.loc[song_indices, 'genre']

    precision = sum(recommended_genres == genre_input) / k
    return precision

precision_at_k('Shape of You', k=5)

0.4


Untuk mengevaluasi kualitas rekomendasi, kita menggunakan metrik **Precision@K**.

Precision@K mengukur seberapa banyak lagu yang direkomendasikan dari genre yang sama dengan lagu input, dari total K rekomendasi.

Contoh implementasi fungsi `precision_at_k`:

```python
def precision_at_k(song_name, k=5):
    song_name = song_name.lower()
    if song_name not in track_to_idx:
        return f"Lagu '{song_name}' tidak ditemukan."
    
    idx = track_to_idx[song_name]
    genre_input = df.loc[idx, 'genre']
    
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:k+1]  # exclude lagu itu sendiri
    
    song_indices = [i[0] for i in sim_scores]
    recommended_genres = df.loc[song_indices, 'genre']
    
    precision = sum(recommended_genres == genre_input) / k
    return precision